In [1]:
from sklearn import datasets
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
import numpy as np
import os
from torchvision.datasets import ImageFolder
import torch.nn as nn
import timm
from torchvision import models
import torch.optim as optim
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

# Set dataset path
dataset_path = "/Deepfake/code/FF++/"
train_dataset_path = os.path.join(dataset_path, "Train")
test_dataset_path = os.path.join(dataset_path, "Test")

# Load datasets
full_dataset = ImageFolder(root=train_dataset_path)
test_dataset = ImageFolder(root=test_dataset_path)

# Define transformations
IMG_SIZE = 224
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),  # Random rotation
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random affine transformation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Apply transforms
full_dataset.transform = train_transforms
test_dataset.transform = val_transforms

# Split dataset
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transforms

BATCH_SIZE = 8
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# Model Architecture with Ensemble
class EnsembleDeepfakeDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.effnet = models.efficientnet_b1(weights=models.EfficientNet_B1_Weights.DEFAULT)
        self.effnet.classifier = nn.Identity()  # Remove the classifier
        self.vit = timm.create_model("vit_base_patch16_224", pretrained=True)
        self.vit.head = nn.Identity()  # Remove the head
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.resnet.fc = nn.Identity()  # Remove the final fully connected layer
        
        # Calculate the output sizes
        effnet_output_size = 1280  # EfficientNet-B1 output size
        vit_output_size = 768  # ViT output size
        resnet_output_size = 2048  # ResNet50 output size
        
        # Adjust the input size for the classifier
        self.classifier = nn.Sequential(
            nn.Linear(effnet_output_size + vit_output_size + resnet_output_size, 128),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        eff_features = self.effnet(x)
        vit_features = self.vit(x)
        resnet_features = self.resnet(x)
        combined_features = torch.cat([eff_features, vit_features, resnet_features], dim=1)
        return self.classifier(combined_features)

# Device, model, loss, optimizer
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EnsembleDeepfakeDetector().to(DEVICE)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=2)
scaler = torch.cuda.amp.GradScaler()

# Resume training if checkpoint exists
checkpoint_path = "checkpoint_epoch_9.pth"
start_epoch = 10
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scaler.load_state_dict(checkpoint['scaler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"Resumed training from epoch {start_epoch}")
else:
    print("No checkpoint found. Starting fresh training.")

# Training Loop
EPOCHS = 20
for epoch in range(start_epoch, EPOCHS):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.float().to(DEVICE).unsqueeze(1)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast("cuda"):
            outputs = model(images)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item() * images.size(0)
        torch.cuda.empty_cache()
    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_loss / len(train_loader.dataset):.4f}")
    scheduler.step(train_loss)

    # Save checkpoint
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler_state_dict': scaler.state_dict(),
    }, f'checkpoint_epoch_{epoch}.pth')

# Model Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            preds = torch.sigmoid(outputs).cpu().numpy().flatten()
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(true_labels, np.round(predictions))
    auc = roc_auc_score(true_labels, predictions)
    f1 = f1_score(true_labels, np.round(predictions))
    precision = precision_score(true_labels, np.round(predictions))
    recall = recall_score(true_labels, np.round(predictions))
    return accuracy, auc, f1, precision, recall

accuracy, auc, f1, precision, recall = evaluate_model(model, test_loader)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test AUC: {auc:.4f}")
print(f"Test F1 Score: {f1:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")

# Predict single image
#def predict(image_path, model):
    #model.eval()
   # image = Image.open(image_path).convert("RGB")
   # image = val_transforms(image).unsqueeze(0).to(DEVICE)
   # with torch.no_grad():
   #     output = model(image)
   #     prediction = torch.sigmoid(output).item()
   # return "Fake" if prediction > 0.5 else "Real"

# Example single image
#print(predict("/content/testimage_deepfake.jpg", model))

# Predict multiple images
def predict_multiple(images_dir, model):
    model.eval()
    results = {}
    for filename in os.listdir(images_dir):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(images_dir, filename)
            try:
                image = Image.open(image_path).convert("RGB")
                image = val_transforms(image).unsqueeze(0).to(DEVICE)
                with torch.no_grad():
                    output = model(image)
                    prediction = torch.sigmoid(output).item()
                label = "Fake" if prediction > 0.5 else "Real"
                results[filename] = (label, round(prediction, 4))
            except Exception as e:
                results[filename] = f"Error: {e}"
    return results

# Example multiple image usage
image_folder_path = "/content/multiple_test_images/"
predictions = predict_multiple(image_folder_path, model)
for image_name, result in predictions.items():
    print(f"{image_name}: {result}")

Training samples: 2012
Validation samples: 504
Test samples: 3552


C:\Users\A\AppData\Local\Temp\ipykernel_9916\1317453136.py:99: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
c:\Users\A\anaconda3\Lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Resumed training from epoch 10


C:\Users\A\AppData\Local\Temp\ipykernel_9916\1317453136.py:122: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast("cuda"):
c:\Users\A\anaconda3\Lib\site-packages\torch\amp\autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Epoch 11/20 - Train Loss: 0.0064
Epoch 12/20 - Train Loss: 0.0014
Epoch 13/20 - Train Loss: 0.0012
Epoch 14/20 - Train Loss: 0.0021
Epoch 15/20 - Train Loss: 0.0006
Epoch 16/20 - Train Loss: 0.0010
Epoch 17/20 - Train Loss: 0.0002
Epoch 18/20 - Train Loss: 0.0023
Epoch 19/20 - Train Loss: 0.0010
Epoch 20/20 - Train Loss: 0.0004
Test Accuracy: 0.9091
Test AUC: 0.9995
Test F1 Score: 0.8950
Test Precision: 0.8099
Test Recall: 1.0000


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/multiple_test_images/'

In [1]:
from sklearn import datasets
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
import numpy as np
import os
from torchvision.datasets import ImageFolder
import torch.nn as nn
import timm
from torchvision import models
import torch.optim as optim
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

# Set dataset path
dataset_path = "/Deepfake/code/FF++/"
train_dataset_path = os.path.join(dataset_path, "Train")
test_dataset_path = os.path.join(dataset_path, "Test")

# Load datasets
full_dataset = ImageFolder(root=train_dataset_path)
test_dataset = ImageFolder(root=test_dataset_path)

# Define transformations
IMG_SIZE = 224
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),  # Random rotation
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random affine transformation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Apply transforms
full_dataset.transform = train_transforms
test_dataset.transform = val_transforms

# Split dataset
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transforms

BATCH_SIZE = 8
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# Model Architecture with Ensemble
class EnsembleDeepfakeDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.effnet = models.efficientnet_b1(weights=models.EfficientNet_B1_Weights.DEFAULT)
        self.effnet.classifier = nn.Identity()  # Remove the classifier
        self.vit = timm.create_model("vit_base_patch16_224", pretrained=True)
        self.vit.head = nn.Identity()  # Remove the head
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.resnet.fc = nn.Identity()  # Remove the final fully connected layer
        
        # Calculate the output sizes
        effnet_output_size = 1280  # EfficientNet-B1 output size
        vit_output_size = 768  # ViT output size
        resnet_output_size = 2048  # ResNet50 output size
        
        # Adjust the input size for the classifier
        self.classifier = nn.Sequential(
            nn.Linear(4096, 128),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.GELU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        eff_features = self.effnet(x)
        vit_features = self.vit(x)
        resnet_features = self.resnet(x)
        combined_features = torch.cat([eff_features, vit_features, resnet_features], dim=1)
        return self.classifier(combined_features)

# Device, model, loss, optimizer
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EnsembleDeepfakeDetector().to(DEVICE)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=2)
scaler = torch.cuda.amp.GradScaler()

# Resume training if checkpoint exists
checkpoint_path = "checkpoint_epoch_13.pth"
start_epoch = 14
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scaler.load_state_dict(checkpoint['scaler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"Resumed training from epoch {start_epoch}")
else:
    print("No checkpoint found. Starting fresh training.")

# Training Loop
EPOCHS = 20
for epoch in range(start_epoch, EPOCHS):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.float().to(DEVICE).unsqueeze(1)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast("cuda"):
            outputs = model(images)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item() * images.size(0)
        torch.cuda.empty_cache()
    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_loss / len(train_loader.dataset):.4f}")
    scheduler.step(train_loss)

    # Save checkpoint
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler_state_dict': scaler.state_dict(),
    }, f'checkpoint_epoch_{epoch}.pth')

# Model Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            preds = torch.sigmoid(outputs).cpu().numpy().flatten()
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(true_labels, np.round(predictions))
    auc = roc_auc_score(true_labels, predictions)
    f1 = f1_score(true_labels, np.round(predictions))
    precision = precision_score(true_labels, np.round(predictions))
    recall = recall_score(true_labels, np.round(predictions))
    return accuracy, auc, f1, precision, recall

accuracy, auc, f1, precision, recall = evaluate_model(model, test_loader)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test AUC: {auc:.4f}")
print(f"Test F1 Score: {f1:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")



# Predict multiple images
def predict_multiple(images_dir, model):
    model.eval()
    results = {}
    for filename in os.listdir(images_dir):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(images_dir, filename)
            try:
                image = Image.open(image_path).convert("RGB")
                image = val_transforms(image).unsqueeze(0).to(DEVICE)
                with torch.no_grad():
                    output = model(image)
                    prediction = torch.sigmoid(output).item()
                label = "Fake" if prediction > 0.5 else "Real"
                results[filename] = (label, round(prediction, 4))
            except Exception as e:
                results[filename] = f"Error: {e}"
    return results

# Example multiple image usage
image_folder_path = "/Deepfake/code/Multiple_outputs"
predictions = predict_multiple(image_folder_path, model)
for image_name, result in predictions.items():
    print(f"{image_name}: {result}")

Training samples: 2012
Validation samples: 504
Test samples: 3552


C:\Users\A\AppData\Local\Temp\ipykernel_18620\2277575716.py:102: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
c:\Users\A\anaconda3\Lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Resumed training from epoch 14


C:\Users\A\AppData\Local\Temp\ipykernel_18620\2277575716.py:125: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast("cuda"):
c:\Users\A\anaconda3\Lib\site-packages\torch\amp\autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Epoch 15/20 - Train Loss: 0.0234
Epoch 16/20 - Train Loss: 0.0556
Epoch 17/20 - Train Loss: 0.0260
Epoch 18/20 - Train Loss: 0.0128
Epoch 19/20 - Train Loss: 0.0296
Epoch 20/20 - Train Loss: 0.0324
Test Accuracy: 0.9578
Test AUC: 1.0000
Test F1 Score: 0.9483
Test Precision: 0.9017
Test Recall: 1.0000


In [1]:
from sklearn import datasets
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
import numpy as np
import os
from torchvision.datasets import ImageFolder
import torch.nn as nn
import timm
from torchvision import models
import torch.optim as optim
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score

# Set dataset path
dataset_path = "/Deepfake/code/Combined(FF+&Celeb-DF)"
train_dataset_path = os.path.join(dataset_path, "Train")
test_dataset_path = os.path.join(dataset_path, "Test")

# Load datasets
full_dataset = ImageFolder(root=train_dataset_path)
test_dataset = ImageFolder(root=test_dataset_path)

# Define transformations
IMG_SIZE = 224
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),  # Random rotation
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random affine transformation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Apply transforms
full_dataset.transform = train_transforms
test_dataset.transform = val_transforms

# Split dataset
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transforms

BATCH_SIZE = 8
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")

# Model Architecture with Ensemble
class EnsembleDeepfakeDetector(nn.Module):
    def __init__(self):
        super().__init__()
        self.effnet = models.efficientnet_b1(weights=models.EfficientNet_B1_Weights.DEFAULT)
        self.effnet.classifier = nn.Identity()  # Remove the classifier
        self.vit = timm.create_model("vit_base_patch16_224", pretrained=True)
        self.vit.head = nn.Identity()  # Remove the head
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.resnet.fc = nn.Identity()  # Remove the final fully connected layer
        
        # Calculate the output sizes
        effnet_output_size = 1280  # EfficientNet-B1 output size
        vit_output_size = 768  # ViT output size
        
        # Adjust the input size for the classifier
        self.classifier = nn.Sequential(
            nn.Linear(4096, 128),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.GELU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        eff_features = self.effnet(x)
        vit_features = self.vit(x)
        resnet_features = self.resnet(x)
        combined_features = torch.cat([eff_features, vit_features, resnet_features], dim=1)
        return self.classifier(combined_features)

# Device, model, loss, optimizer
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EnsembleDeepfakeDetector().to(DEVICE)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=2)
scaler = torch.cuda.amp.GradScaler()

# Resume training if checkpoint exists
checkpoint_path = "checkpoint_epoch_0.pth"
start_epoch = 0
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scaler.load_state_dict(checkpoint['scaler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"Resumed training from epoch {start_epoch}")
else:
    print("No checkpoint found. Starting fresh training.")

# Training Loop
EPOCHS = 20
for epoch in range(start_epoch, EPOCHS):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.float().to(DEVICE).unsqueeze(1)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast("cuda"):
            outputs = model(images)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item() * images.size(0)
        torch.cuda.empty_cache()
    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_loss / len(train_loader.dataset):.4f}")
    scheduler.step(train_loss)

    # Save checkpoint
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler_state_dict': scaler.state_dict(),
    }, f'checkpoint_epoch_{epoch}.pth')

# Model Evaluation
def evaluate_model(model, test_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            preds = torch.sigmoid(outputs).cpu().numpy().flatten()
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(true_labels, np.round(predictions))
    auc = roc_auc_score(true_labels, predictions)
    f1 = f1_score(true_labels, np.round(predictions))
    precision = precision_score(true_labels, np.round(predictions))
    recall = recall_score(true_labels, np.round(predictions))
    return accuracy, auc, f1, precision, recall

accuracy, auc, f1, precision, recall = evaluate_model(model, test_loader)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test AUC: {auc:.4f}")
print(f"Test F1 Score: {f1:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")



# Predict multiple images
def predict_multiple(images_dir, model):
    model.eval()
    results = {}
    for filename in os.listdir(images_dir):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(images_dir, filename)
            try:
                image = Image.open(image_path).convert("RGB")
                image = val_transforms(image).unsqueeze(0).to(DEVICE)
                with torch.no_grad():
                    output = model(image)
                    prediction = torch.sigmoid(output).item()
                label = "Fake" if prediction > 0.5 else "Real"
                results[filename] = (label, round(prediction, 4))
            except Exception as e:
                results[filename] = f"Error: {e}"
    return results

# Example multiple image usage
image_folder_path = "/Deepfake/code/Multiple_outputs"
predictions = predict_multiple(image_folder_path, model)
for image_name, result in predictions.items():
    print(f"{image_name}: {result}")

Training samples: 9306
Validation samples: 2327
Test samples: 2400


C:\Users\A\AppData\Local\Temp\ipykernel_19764\1309443302.py:101: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
c:\Users\A\anaconda3\Lib\site-packages\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
C:\Users\A\AppData\Local\Temp\ipykernel_19764\1309443302.py:124: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast("cuda"):
c:\Users\A\anaconda3\Lib\site-packages\torch\amp\autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


No checkpoint found. Starting fresh training.


KeyboardInterrupt: 